In [63]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

# 영화감상리뷰로 train

In [4]:
train_df = pd.read_csv('/content/ratings_train.txt', encoding='utf8', sep='\t')
train_df

id                                           document  label
0        9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1        3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2       10265843                                  너무재밓었다그래서보는것을추천한다      0
3        9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4        6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
...          ...                                                ...    ...
149995   6222902                                인간이 문제지.. 소는 뭔죄인가..      0
149996   8549745                                      평점이 너무 낮아서...      1
149997   9311800                    이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?      0
149998   2376369                        청춘 영화의 최고봉.방황과 우울했던 날들의 자화상      1
149999   9619869                           한국 영화 최초로 수간하는 내용이 담긴 영화      0

[150000 rows x 3 columns]

In [5]:
# info
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [6]:
# notnull 제거
train_df = train_df[train_df['document'].notnull()]
train_df

id                                           document  label
0        9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1        3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2       10265843                                  너무재밓었다그래서보는것을추천한다      0
3        9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4        6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
...          ...                                                ...    ...
149995   6222902                                인간이 문제지.. 소는 뭔죄인가..      0
149996   8549745                                      평점이 너무 낮아서...      1
149997   9311800                    이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?      0
149998   2376369                        청춘 영화의 최고봉.방황과 우울했던 날들의 자화상      1
149999   9619869                           한국 영화 최초로 수간하는 내용이 담긴 영화      0

[149995 rows x 3 columns]

In [7]:
# label 0, 1 각각의 개수
train_df['label'].value_counts()

0    75170
1    74825
Name: label, dtype: int64

In [8]:
# 규칙을 가지고 필터링해주는 라이브러리
import re

In [9]:
# re.sub: 대체
# re.sub(r'[^ㄱ-ㅣ가-힣]+'): 정규식 필터링 - 한글 제외한 나머지 문자 공백으로 처리
train_df['document'] = train_df['document'].apply(lambda x: re.sub(r'[^ㄱ-ㅣ가-힣]+', "", x))

In [10]:
train_df.head()

id                                           document  label
0   9976970                                      아더빙진짜짜증나네요목소리      0
1   3819312                            흠포스터보고초딩영화줄오버연기조차가볍지않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                               교도소이야기구먼솔직히재미는없다평점조정      0
4   6483659  사이몬페그의익살스런연기가돋보였던영화스파이더맨에서늙어보이기만했던커스틴던스트가너무나도이...      1

In [11]:
# !pip install konlpy

     |████████████████████████████████| 19.4 MB 2.1 MB/s 
     |████████████████████████████████| 86 kB 2.6 MB/s 
     |████████████████████████████████| 448 kB 35.6 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [12]:
from konlpy.tag import Okt

In [13]:
okt = Okt()

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
tfidf = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1, 2), min_df=3, max_df=0.9)
# tokenizer=okt.morphs : 
# ngram_range(1, 2) : 2개짜리까지 허용
# min_df=3 : 출현 빈도는 최소 3번 이상
# max_df=0.9 : 최대 90% 이하인 것만 사용

tfidf

TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                tokenizer=<bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x7fd5d5d10150>>)

In [16]:
tfidf.fit(train_df['document'])

TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                tokenizer=<bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x7fd5d5d10150>>)

In [25]:
train_tfidf_df = tfidf.transform(train_df['document'])
train_tfidf_df

<149995x114701 sparse matrix of type '<class 'numpy.float64'>'
	with 2579334 stored elements in Compressed Sparse Row format>

# 영화감상리뷰 test

In [26]:
test_df = pd.read_csv('/content/ratings_test.txt', encoding='utf8', sep='\t')
test_df

id                                           document  label
0      6270596                                                굳 ㅋ      1
1      9274899                               GDNTOPCLASSINTHECLUB      0
2      8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3      6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4      6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0
...        ...                                                ...    ...
49995  4608761          오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함      1
49996  5308387       의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO      0
49997  9072549                 그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다      0
49998  5802125     절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네      0
49999  6070594                                         마무리는 또 왜이래      0

[50000 rows x 3 columns]

In [27]:
# notnull 제거
test_df = test_df[test_df['document'].notnull()]
test_df

id                                           document  label
0      6270596                                                굳 ㅋ      1
1      9274899                               GDNTOPCLASSINTHECLUB      0
2      8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3      6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4      6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0
...        ...                                                ...    ...
49995  4608761          오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함      1
49996  5308387       의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO      0
49997  9072549                 그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다      0
49998  5802125     절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네      0
49999  6070594                                         마무리는 또 왜이래      0

[49997 rows x 3 columns]

In [28]:
# re.sub: 대체
# re.sub(r'[^ㄱ-ㅣ가-힣]+'): 정규식 필터링 - 한글 제외한 나머지 문자 공백으로 처리
test_df['document'] = test_df['document'].apply(lambda x: re.sub(r'[^ㄱ-ㅣ가-힣]+', "", x))
test_df

id                           document  label
0      6270596                                 굳ㅋ      1
1      9274899                                         0
2      8544678           뭐야이평점들은나쁘진않지만점짜리는더더욱아니잖아      0
3      6825595               지루하지는않은데완전막장임돈주고보기에는      0
4      6723715      만아니었어도별다섯개줬을텐데왜로나와서제심기를불편하게하죠      0
...        ...                                ...    ...
49995  4608761  오랜만에평점로긴했네ㅋㅋ킹왕짱쌈뽕한영화를만났습니다강렬하게육쾌함      1
49996  5308387     의지박약들이나하는거다탈영은일단주인공김대희닮았고이등병찐따      0
49997  9072549           그림도좋고완성도도높았지만보는내내불안하게만든다      0
49998  5802125     절대봐서는안될영화재미도없고기분만잡치고한세트장에서다해먹네      0
49999  6070594                           마무리는또왜이래      0

[49997 rows x 3 columns]

In [33]:
test_tfidf_df = tfidf.transform(test_df['document'])
test_tfidf_df

<49997x114701 sparse matrix of type '<class 'numpy.float64'>'
	with 829481 stored elements in Compressed Sparse Row format>

# Logistic Regression

In [24]:
from sklearn.linear_model import LogisticRegression

SA_lr = LogisticRegression(random_state = 0)

In [34]:
SA_lr.fit(train_tfidf_df, train_df['label'])

LogisticRegression(random_state=0)

In [35]:
test_predict = SA_lr.predict(test_tfidf_df)

In [36]:
from sklearn.metrics import accuracy_score

print("감성 분석 정확도: ", round(accuracy_score(test_df['label'], test_predict), 3))

감성 분석 정확도:  0.849


In [46]:
# json file load
with open('/content/코로나_naver_news.json', encoding='utf8') as j_f:
  data = json.load(j_f)
data

[{'cnt': 1,
  'description': '<b>코로나</b>발 경제 위기 대응을 위해 돈 쓸 곳은 늘어났지만, 국세 수입은 줄어들면서 정부의 재정 마련에 대한 우려가 컸다. 이 때문에 한국개발원(KDI) 등 국책연구기관들은 증세를 화두로 꺼내들었지만, 정부 여당은 증세에... ',
  'pDate': '2020-06-04 14:12:00',
  'title': "결국 '증세론' 먼저 꺼내든 與…&quot;증세없는 '기본소득' 불가능&quot;"},
 {'cnt': 2,
  'description': '▲ 지난 2일 창녕군보건소 앞에 설치한 선별진료소에서 검사자가 체온을 측정하고 있다.©(사진제공=창녕군청) <b>코로나</b> 장기화 대비 비대면 선별진료 도입 경남 창녕군은 지난 2월 28일 도내 최초로 <b>코로나</b>19 선별진료소... ',
  'pDate': '2020-06-04 14:12:00',
  'title': '창녕군, ‘창녕형’비대면 선별진료소 운영'},
 {'cnt': 3,
  'description': "한편, ‘설현’은 최근 <b>코로나</b>19 바이러스를 다룬, MBC ‘시리즈M’ PANDEMIC(세계적 유행)에서 다큐멘터리 내레이션에 처음 도전해 호평을 받았으며, tvN의 새 드라마 '낮과 밤' 출연을 검토중이다. 게임과 이벤트에 대한... ",
  'pDate': '2020-06-04 14:12:00',
  'title': "DK모바일, 메인 홍보 모델로 AOA '설현' 선정"},
 {'cnt': 4,
  'description': '변화의 핵심 중에서는 우리 사회가 신종 <b>코로나</b>바이러스 감염증(<b>코로나</b>19)의 위기를 마주하고 있다. 언제 끝이날 지 또 앞으로 어떠한 미래가 펼치질 지 모른다. 국가가 나서서 국민의 어려움을 해결해 줘야 한다. 저희... ',
  'pDate': '2020-06-04 14:12:00',
  'title': '김병민 &quot;기본소득도 필요하면 논의 테이블에 올려야

In [48]:
# 데이터프레임으로 변환
news_df = pd.DataFrame(data)
news_df

cnt  ...                                              title
0      1  ...      결국 '증세론' 먼저 꺼내든 與…&quot;증세없는 '기본소득' 불가능&quot;
1      2  ...                             창녕군, ‘창녕형’비대면 선별진료소 운영
2      3  ...                       DK모바일, 메인 홍보 모델로 AOA '설현' 선정
3      4  ...       김병민 &quot;기본소득도 필요하면 논의 테이블에 올려야&quot; [인터뷰]
4      5  ...                            이재갑 장관, 고용안정지원금 서울센터 방문
..   ...  ...                                                ...
795  796  ...  [세계의 눈] &quot;<b>코로나</b>19 영향, 현대차 매출 큰 타격&quo...
796  797  ...                   중부발전, 소규모 태양광사업자 REC 판매대금 '선 지급'
797  798  ...  &quot;<b>코로나</b> 함께 극복&quot;…박종환 자유총연맹 총재, 캠페인 참여
798  799  ...             치과의사 7000명 모이는 행사 앞두고 치협 회장 “행사 취소해달라”
799  800  ...                        [IS포토|대종상영화제] 문가영, '발랄 듬~뿍'

[800 rows x 4 columns]

In [49]:
# notnull 확인
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cnt          800 non-null    int64 
 1   description  800 non-null    object
 2   pDate        800 non-null    object
 3   title        800 non-null    object
dtypes: int64(1), object(3)
memory usage: 25.1+ KB


In [53]:
# 정규식으로 특수문자 필터링
st2 = []
for st in news_df['title']:
  st2.append(re.sub(r'[^ ㄱ-ㅎ|가-힣]+', "", st))
st2

['결국 증세론 먼저 꺼내든 증세없는 기본소득 불가능',
 '창녕군 창녕형비대면 선별진료소 운영',
 '모바일 메인 홍보 모델로  설현 선정',
 '김병민 기본소득도 필요하면 논의 테이블에 올려야 인터뷰',
 '이재갑 장관 고용안정지원금 서울센터 방문',
 '부산행반도로 이어지는 한국형 좀비 세계관 칸의 선택 받았다',
 '년만 첫 불허에도 홍콩 톈안문 추모 촛불',
 '국내 대 항만 월 물동량 인천항만 상승곡선',
 '경기도의회 농정해양위 농민기본소득 도입 이재명 지사와 집행부에 강력 촉구',
 '이용빈 원내부대표 코로나 대응 의료진노동자 건강권 보장해야',
 '코로나로 인해 변한 원광한의대 경혈학 실습 풍경',
 '경산시 농산물 드라이브 스루 특별할인 판매행사 개최',
 '당신이 아는 미술 시장이 아는 미술  한국 현대 미술 시장을 이끄는 작가',
 '제회 부산국제영화제 공식 상영작 모집 공식',
 '수도권 코로나 확진자 산발적으로 늘어하루 새 명 증가',
 '동작구 전통시장 무료 배송서비스 사업 실시',
 '서울 건강용품 세미나서 무더기 확진명 검사',
 ' 상반기 신입 및 경력직 공채 시행',
 '중국 굴착기 판매량 상승에도두산인프라코어 긴장하는 이유는',
 '미소짓는 의료진',
 '포스트 코로나 대비투자 잰걸음 나서는 게임사',
 '휴넷 리더십 게임러닝 프로그램 아르고 출시',
 '광주 서구 양동 민관 협력으로 주거환경 개선',
 '손흥민의 고백 주간의 군사훈련 힘들지만 즐겼다',
 ' 카타르 선 수주 문 대통령 경제외교 결실',
 '유통나누기 세상더하기 세븐일레븐 유치원 통학차량에 공기청정기 설치',
 '김포 혁신교육지구 사업 특수교육대상자를 위한 교육과정 지원',
 '속보 군포시청 서울시 리치웨이 코로나 확진자 동선 공개 군포로번길',
 '가맹점과 상생나선 카드사',
 '질병관리본부 청 승격에도 줄어든 입지복지부 전문성독립성 확보가',
 '함양군 브리핑 명품 함양양파 특판 서울서 인기몰이 등',
 '이십세기 민아 룰라 때문에 다른 가수들 앨범 일정

In [54]:
# st column 생성
news_df['st'] = st2
news_df

cnt  ...                                     st
0      1  ...            결국 증세론 먼저 꺼내든 증세없는 기본소득 불가능
1      2  ...                    창녕군 창녕형비대면 선별진료소 운영
2      3  ...                   모바일 메인 홍보 모델로  설현 선정
3      4  ...         김병민 기본소득도 필요하면 논의 테이블에 올려야 인터뷰
4      5  ...                 이재갑 장관 고용안정지원금 서울센터 방문
..   ...  ...                                    ...
795  796  ...  세계의 눈 코로나 영향 현대차 매출 큰 타격 하지만  차량은 순항 
796  797  ...             중부발전 소규모 태양광사업자  판매대금 선 지급
797  798  ...           코로나 함께 극복박종환 자유총연맹 총재 캠페인 참여
798  799  ...       치과의사 명 모이는 행사 앞두고 치협 회장 행사 취소해달라
799  800  ...                    포토|대종상영화제 문가영 발랄 듬뿍

[800 rows x 5 columns]

In [55]:
# input받은 str을 list로
st_tfidf_df = tfidf.transform(list(news_df['st']))
st_tfidf_df

<800x114701 sparse matrix of type '<class 'numpy.float64'>'
	with 6130 stored elements in Compressed Sparse Row format>

In [56]:
# 부정/긍정 예측
st_predict = SA_lr.predict(st_tfidf_df)
st_predict

array([0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,

In [73]:
num_pos = np.count_nonzero(st_predict)
num_pos

289

In [78]:
if num_pos <= len(st_predict)/2:
  print("에잉,,ㅉㅉ,, 오늘은 망했군!")
else:
  print("내일도 좋은일만 :)")

에잉,,ㅉㅉ,, 오늘은 망했군!
